In [ ]:
!wget --header="Host: doc-0o-7c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_gbsevnc13krov7rjgq2cj0s062jqolpo=06568700698331676125|1634615700000|klej5cj5jrhqi9vkvioj16q2bmde14in" --header="Connection: keep-alive" "https://doc-0o-7c-docs.googleusercontent.com/docs/securesc/ths62md7879q0rdb5g663rm4c96q2241/sr1phqn2ct71pa1m2jr75p8755ogd8g1/1634615700000/00484516897554883881/06568700698331676125/1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_?e=download&authuser=0" -c -O 'glove_vectors'

--2021-10-19 03:56:43--  https://doc-0o-7c-docs.googleusercontent.com/docs/securesc/ths62md7879q0rdb5g663rm4c96q2241/sr1phqn2ct71pa1m2jr75p8755ogd8g1/1634615700000/00484516897554883881/06568700698331676125/1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_?e=download&authuser=0
Resolving doc-0o-7c-docs.googleusercontent.com (doc-0o-7c-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-0o-7c-docs.googleusercontent.com (doc-0o-7c-docs.googleusercontent.com)|172.217.204.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127506004 (122M) [application/octet-stream]
Saving to: ‘glove_vectors’

glove_vectors       100%[===================>] 121.60M  97.7MB/s    in 1.2s    

2021-10-19 03:56:45 (97.7 MB/s) - ‘glove_vectors’ saved [127506004/127506004]



# Import Libraries

In [ ]:
import scipy
import pandas as pd
import numpy as np
from numpy import zeros
import nltk
import pickle
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as sp_randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics  import f1_score,roc_auc_score
import warnings
nltk.download('vader_lexicon')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
#read train and validation data
train_data=pd.read_csv('/content/drive/MyDrive/Meme/data/train_data.csv')
validation_data=pd.read_csv("/content/drive/MyDrive/Meme/data/validation_data.csv")
test_data=pd.read_csv('/content/drive/MyDrive/Meme/data/test_data.csv')

In [ ]:
#fill thumbnail.width missing values
mean_width=train_data['thumbnail.width'].mean()
train_data['thumbnail.width'].fillna(value=mean_width,inplace=True)
validation_data['thumbnail.width'].fillna(value=mean_width,inplace=True)
test_data['thumbnail.width'].fillna(value=mean_width,inplace=True)

# fill  thumbnail.height missing values
mean_height=train_data['thumbnail.height'].mean()
train_data['thumbnail.height'].fillna(value=mean_height,inplace=True)
validation_data['thumbnail.height'].fillna(value=mean_height,inplace=True)
test_data['thumbnail.height'].fillna(value=mean_height,inplace=True)


In [ ]:
#calculate area of image
train_data['area']=train_data['thumbnail.height']*train_data['thumbnail.width']
validation_data['area']=validation_data['thumbnail.height']*validation_data['thumbnail.width']

In [ ]:
#define sentiment scores on text
def sentiment_scores(data):
    sid = SentimentIntensityAnalyzer()
    neg=[]
    pos=[]
    neu=[]
    comp=[]
    for sentence in data['text'].values: 
        sentence_sentiment_score = sid.polarity_scores(sentence)
        comp.append(sentence_sentiment_score['compound'])
        neg.append(sentence_sentiment_score['neg'])
        pos.append(sentence_sentiment_score['pos'])
        neu.append(sentence_sentiment_score['neu'])
    return comp,neg,pos,neu

# Train data  sentiment_scores
train_comp,train_neg,train_pos,train_neu=sentiment_scores(train_data)
train_data['com']=train_comp
train_data['neg']=train_neg
train_data['pos']=train_pos
train_data['neu']=train_neu

#validation data sentiment_scores
validation_comp,validation_neg,validation_pos,validation_neu=sentiment_scores(validation_data)
validation_data['com']=validation_comp
validation_data['neg']=validation_neg
validation_data['pos']=validation_pos
validation_data['neu']=validation_neu

# test data sentiment_scores
test_comp,test_neg,test_pos,test_neu=sentiment_scores(test_data)
test_data['com']=test_comp
test_data['neg']=test_neg
test_data['pos']=test_pos
test_data['neu']=test_neu

Read Image features file

In [ ]:
train_image_features=pd.read_csv('/content/drive/MyDrive/Meme/data/train_hsv_data.csv')
train_image_features['id']=train_image_features['id'].apply(lambda x:x.split(".")[0])
train_image_features.head(4)

,id,avg_red,avg_green,avg_blue,avg_h,avg_s,avg_v
0,fkzefu,239.172259,238.365883,238.055094,6.113741,2.055231,239.278907
1,fkz38w,167.921128,154.234400,146.326260,39.118095,52.000453,169.767484
2,fkz4wb,67.998585,80.056550,78.199143,35.447353,33.416639,81.083201
3,fkywwv,70.633591,68.253622,65.874174,79.060371,98.429703,81.938204


In [ ]:
validation_image_features=pd.read_csv('/content/drive/MyDrive/Meme/data/validation_hsv_data.csv')
validation_image_features['id']=validation_image_features['id'].apply(lambda x:x.split(".")[0])
validation_image_features.head(4)

,id,avg_red,avg_green,avg_blue,avg_h,avg_s,avg_v
0,fi8zos,105.825954,95.935978,84.807401,18.997557,52.505151,105.987770
1,fie2qx,116.319292,112.056747,103.298224,14.058189,43.652817,116.648814
2,fidedo,167.300836,165.844258,167.075575,39.674466,20.069788,170.325628
3,fipe1b,114.096877,109.441654,101.716981,36.235294,41.504065,114.883887


Merge Image Features

In [ ]:
train_data=train_data.merge(train_image_features,on='id',how='left')
print(train_data.shape)
train_data.head(3)

(3264, 42)


,id,ocr_text,ocr_no_of_words,ocr_text_len,ocr_preprocess,ocr_preprocess_no_of_words,ocr_preprocess_text_len,index,created_utc,downs,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url,datetime_temp,time_of_day,ups_normed,dank_level,title_no_of_words,title_len,title_preprocess,title_preprocess_no_of_words,title_preprocess_text_len,path,text,area,com,neg,pos,neu,avg_red,avg_green,avg_blue,avg_h,avg_s,avg_v
0,fjn3gy,when acvedenalh bite their toungue The monster...,11.0,65.0,bite monster become,3.0,19.0,28606.0,1.584374e+09,0.0,False,https://i.redd.it/swm384fj32n41.jpg,r/memes,9552833.0,140.0,https://b.thumbs.redditmedia.com/C15Fr3fcu3dln...,140.0,Absolutely disgusting,975.0,/r/memes/comments/fjn3gy/absolutely_disgusting/,2020-03-16 10:51:43,2.0,1.020640e-04,1.0,2.0,21.0,absolutely disgusting,2.0,21.0,/content/train/class_1/fjn3gy.jpg,absolutely disgusting bite monster become,19600.0,-0.5709,0.48,0.00,0.52,154.929228,137.958439,137.032199,42.276996,52.257750,155.450549
1,fjn514,42069 OUNLMATU PAHDOMORO 12 ;,5.0,29.0,NaN,0.0,0.0,28575.0,1.584374e+09,0.0,False,https://i.redd.it/4hdh24op32n41.jpg,r/memes,9552833.0,105.0,default,140.0,Words cannot describe how much happiness I hav...,4.0,/r/memes/comments/fjn514/words_cannot_describe...,2020-03-16 10:54:27,2.0,4.187240e-07,0.0,9.0,57.0,words can not describe how much happiness have...,9.0,56.0,/content/train/class_0/fjn514.jpg,words can not describe how much happiness have...,14700.0,0.5574,0.00,0.31,0.69,174.331474,172.198407,169.971316,33.731953,30.351939,182.356801
2,fjnkyy,Extroverts in quarantine Me an extrovert that ...,11.0,72.0,quarantine extrovert people quarantine,4.0,38.0,28242.0,1.584376e+09,0.0,False,https://i.redd.it/qn2yvbgq82n41.jpg,r/memes,9552832.0,140.0,default,140.0,My first meme,0.0,/r/memes/comments/fjnkyy/my_first_meme/,2020-03-16 11:20:50,2.0,0.000000e+00,0.0,3.0,13.0,my first,2.0,8.0,/content/train/class_0/fjnkyy.jpg,my first quarantine extrovert people quarantine,19600.0,0.0000,0.00,0.00,1.00,86.817131,83.663894,79.729390,11.316440,19.647247,87.663379


In [ ]:
validation_data=validation_data.merge(validation_image_features,on='id',how='left')
print(validation_data.shape)
validation_data.head(3)

(1615, 42)


,id,ocr_text,ocr_no_of_words,ocr_text_len,ocr_preprocess,ocr_preprocess_no_of_words,ocr_preprocess_text_len,index,created_utc,downs,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url,datetime_temp,time_of_day,ups_normed,dank_level,title_no_of_words,title_len,title_preprocess,title_preprocess_no_of_words,title_preprocess_text_len,path,text,area,com,neg,pos,neu,avg_red,avg_green,avg_blue,avg_h,avg_s,avg_v
0,fjswbp,When 4TaFAIR isn't winning against normal name...,17.0,98.0,not winning normal people made wrong feel,7.0,41.0,23658.0,1.584394e+09,0.0,False,https://i.redd.it/sz73uulps3n41.png,r/memes,9552814.0,118.0,https://b.thumbs.redditmedia.com/sHy7W5wGxK69b...,140.0,You can put the sentence into google translate...,18.0,/r/memes/comments/fjswbp/you_can_put_the_sente...,2020-03-16 16:34:35,3.0,0.000002,0.0,12.0,66.0,you can put the sentence into google translate...,12.0,66.0,/content/validation/class_0/fjswbp.jpg,you can put the sentence into google translate...,16520.0,-0.6784,0.254,0.056,0.690,133.694618,120.305931,120.087506,69.180509,98.402620,141.853861
1,fjriyh,Me when everything was closed this weekend: ma...,10.0,61.0,everything closed weekend made,4.0,30.0,24845.0,1.584390e+09,0.0,False,https://i.redd.it/qb77i890e3n41.jpg,r/memes,9552816.0,140.0,https://b.thumbs.redditmedia.com/z0JhDTea1T_uF...,140.0,*Sad me noises*,23.0,/r/memes/comments/fjriyh/sad_me_noises/,2020-03-16 15:12:10,3.0,0.000002,0.0,3.0,15.0,sad me noises,3.0,13.0,/content/validation/class_0/fjriyh.jpg,sad me noises everything closed weekend made,19600.0,-0.4767,0.341,0.000,0.659,144.465904,148.956254,140.275021,58.527916,60.928690,158.697454
2,fjrfvn,"Roses are red, get boned by the pastor; The ch...",19.0,99.0,red get boned pastor fast combine faster,7.0,40.0,24912.0,1.584389e+09,0.0,False,https://i.redd.it/udybjadsc3n41.png,r/memes,9552816.0,140.0,default,140.0,Gotta go speed,2045.0,/r/memes/comments/fjrfvn/gotta_go_speed/,2020-03-16 15:06:59,3.0,0.000214,1.0,3.0,14.0,gotta go speed,3.0,14.0,/content/validation/class_1/fjrfvn.jpg,gotta go speed red get boned pastor fast combi...,19600.0,0.0000,0.000,0.000,1.000,186.236102,171.321177,150.937902,27.600595,78.040035,190.878357


In [ ]:
train_data['hour']=train_data['datetime_temp'].apply(lambda x:int(x[11:13]))
validation_data['hour']=validation_data['datetime_temp'].apply(lambda x:int(x[11:13]))
test_data['hour']=test_data['datetime_temp'].apply(lambda x:int(x[11:13]))

In [ ]:
#numeric features for training
list_features=['ocr_no_of_words', 'ocr_text_len', 'ocr_preprocess_no_of_words', 'ocr_preprocess_text_len',
               'title_no_of_words', 'title_len','title_preprocess_no_of_words', 'title_preprocess_text_len',
              'thumbnail.height' , 'thumbnail.width','time_of_day','hour','area','com', 'neg', 'pos', 'neu',
               'avg_red','avg_green','avg_blue','avg_h','avg_s','avg_v']

def min_max_scaling(df,list_features,dict_mm={ }):
    
    result = df.copy()
    if not bool(dict_mm):  #if dict is empty then we are finding mean and max for train data
        for feature_name in list_features:
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            dict_mm[feature_name]=[max_value,min_value]
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        return result,dict_mm   
    else:                    #use train data min and max
        for feature_name in list_features:
            max_value=dict_mm[feature_name][0]
            min_value=dict_mm[feature_name][1]
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        return result

# min-max scaling on numeric features
train_data,dict_mm=min_max_scaling(train_data,list_features)
validation_data=min_max_scaling(validation_data,list_features,dict_mm)

train_numeric=train_data[list_features]
validation_numeric=validation_data[list_features]


In [ ]:
vectorizer1=TfidfVectorizer(min_df=2,ngram_range=(1,4))
vectorizer1.fit(train_data['text'].values)

train_data_text_tfidf = vectorizer1.transform(train_data['text'].values)
validation_data_text_tfidf = vectorizer1.transform(validation_data['text'].values)
test_data_text_tfidf  = vectorizer1.transform(test_data['text'].values)

print("After vectorizations")
print(train_data_text_tfidf.shape)
print(validation_data_text_tfidf.shape)
print(test_data_text_tfidf.shape)
print("="*100)


After vectorizations
(3264, 7835)
(1615, 7835)
(1637, 7835)


In [ ]:
dictionary = dict(zip(vectorizer1.get_feature_names(), list(vectorizer1.idf_)))
tfidf_words = set(vectorizer1.get_feature_names())

In [ ]:
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [ ]:
# reference appliedai
# average Word2Vec
# compute average word2vec for each review.
tfidf_w2v_vectors_train = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(train_data['text'].values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_train.append(vector)

print(len(tfidf_w2v_vectors_train))


100%|██████████| 3264/3264 [00:00<00:00, 12815.57it/s]

3264


In [ ]:
# average Word2Vec
# compute average word2vec for each review.
tfidf_w2v_vectors_validation = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(validation_data['text'].values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_validation.append(vector)

print(len(tfidf_w2v_vectors_validation))
len(tfidf_w2v_vectors_validation[0])

100%|██████████| 1615/1615 [00:00<00:00, 8654.91it/s]

1615


300

In [ ]:
train_numeric=scipy.sparse.csr_matrix(train_numeric)
validation_numeric=scipy.sparse.csr_matrix(validation_numeric)

In [ ]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

X_train = hstack((tfidf_w2v_vectors_train,   train_numeric)).tocsr()
X_validation = hstack((tfidf_w2v_vectors_validation, validation_numeric)).tocsr()

print("Final Data matrix")
print(X_train.shape)
print(X_validation.shape)
print("="*100)

Final Data matrix
(3264, 323)
(1615, 323)


In [ ]:
X_train = hstack((tfidf_w2v_vectors_train, train_numeric)).tocsr()
X_validation = hstack((tfidf_w2v_vectors_validation, validation_numeric)).tocsr()

print("Final Data matrix")
print(X_train.shape)
print(X_validation.shape)
print("="*100)

Final Data matrix
(3264, 323)
(1615, 323)


In [ ]:
xgbc = XGBClassifier()
parameters = {'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3] ,
              'n_estimators':range(5,100),
              'colsample_bytree':[0.2,0.25,0.3]}
clf =  RandomizedSearchCV(xgbc, parameters, cv=3, scoring='roc_auc',return_train_score=True,n_jobs=-1)
clf.fit(X_train, train_data['dank_level'])


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='de

In [ ]:
results = pd.DataFrame.from_dict(clf.cv_results_)
results[['mean_train_score','mean_test_score', 'param_learning_rate','param_n_estimators','param_colsample_bytree']]

,mean_train_score,mean_test_score,param_learning_rate,param_n_estimators,param_colsample_bytree
0,0.773172,0.605066,0.001,93,0.25
1,0.950858,0.605500,0.1,86,0.2
2,0.738004,0.594600,0.001,16,0.3
3,0.777668,0.597261,0.01,22,0.2
4,0.744495,0.595900,0.01,14,0.3
5,0.778040,0.607070,0.0001,51,0.2
6,0.755468,0.602983,0.0001,74,0.3
7,0.810391,0.597434,0.1,15,0.25
8,0.780161,0.582497,0.3,7,0.25
9,0.756207,0.602234,0.0001,86,0.3


Train xgboost on learning_rate=0.0001 ,n_estimator=74,colsample_bytree=0.3

In [ ]:
xgbc_final = XGBClassifier( learning_rate=0.0001 ,n_estimator=74,colsample_bytree=0.3)
xgbc_final.fit(X_train, train_data['dank_level'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.0001, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimator=74,
              n_estimators=100, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [ ]:
train_prediction=xgbc_final.predict_proba(X_train)
print('train auc is=',roc_auc_score(np.array(train_data['dank_level']), train_prediction[:,1]))
validation_prediction=xgbc_final.predict_proba(X_validation)
print('validation auc is=',roc_auc_score(np.array(validation_data['dank_level']), validation_prediction[:,1]))

train auc is= 0.7177683668870192
validation auc is= 0.6216693517453011
